## This notebook will add movies from year 2018 and 2019

#### We will read the data from tables provided by wikipedia in its article using the link 

In [1]:
import numpy as np
import pandas as pd

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2018"

In [3]:
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [4]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [5]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2019"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [6]:
df = df.append(df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True),ignore_index=True)

In [7]:
df.head()

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2]
1,JANUARY,5,The Strange Ones,Vertical Entertainment,Lauren Wolkstein (director); Christopher Radcl...,[3]
2,JANUARY,5,Stratton,Momentum Pictures,"Simon West (director); Duncan Falconer, Warren...",[4]
3,JANUARY,10,Sweet Country,Samuel Goldwyn Films,"Warwick Thornton (director); David Tranter, St...",[5]
4,JANUARY,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[6]


In [8]:
df.shape

(511, 6)

In [9]:
df.columns

Index(['Opening', 'Opening.1', 'Title', 'Production company', 'Cast and crew',
       'Ref.'],
      dtype='object')

In [10]:
df.drop(["Opening","Opening.1","Production company","Ref."], axis=1, inplace =True)

In [11]:
df.head()

,Title,Cast and crew
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...
2,Stratton,"Simon West (director); Duncan Falconer, Warren..."
3,Sweet Country,"Warwick Thornton (director); David Tranter, St..."
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...


In [12]:
type(df.Title[0])

str

#### Wikipedia doesn't provide genres of movies in that article so we will fetch genres data from a API

TMDB is a free resource to get movies data like its crew member name genres poster overview of any movie. Using python request method we make a call for each movie in out list and update the dataframe with recivied value

In [13]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = "fc671a9683da1e18060546d03e4e88bd"

In [14]:
from tmdbv3api import Movie
tmdb_movie = Movie()
def genre(x):
    genres = []
    result = tmdb_movie.search(x)
    movie_id = result[0].id
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
    data_json = response.json()
    if data_json['genres']:
        genre_str = " " 
        for i in range(0,len(data_json['genres'])):
            genres.append(data_json['genres'][i]['name'])
        return genre_str.join(genres)
    else:
        np.NaN

In [15]:
df["genres"] = df["Title"].map(lambda x: genre(x))

In [16]:
df.head()

,Title,Cast and crew,genres
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Horror Mystery Thriller
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,Thriller Drama
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",Action Thriller
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",Drama History Western
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller


In [17]:
x = df["Cast and crew"][0]
print(x)

Adam Robitel (director); Leigh Whannell (screenplay); Lin Shaye, Angus Sampson, Leigh Whannell, Spencer Locke, Caitlin Gerard, Kirk Acevedo, Bruce Davison


In [18]:
x.split(";")

['Adam Robitel (director)',
 ' Leigh Whannell (screenplay)',
 ' Lin Shaye, Angus Sampson, Leigh Whannell, Spencer Locke, Caitlin Gerard, Kirk Acevedo, Bruce Davison']

In [19]:
x.split(";")[0].split(" (director)")[0]

'Adam Robitel'

In [20]:
def director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [21]:
df["director_name"] = df["Cast and crew"].map(lambda x: director(x))

In [22]:
df.head()

,Title,Cast and crew,genres,director_name
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Horror Mystery Thriller,Adam Robitel
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,Thriller Drama,Lauren Wolkstein
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",Action Thriller,Simon West
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",Drama History Western,Warwick Thornton
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller,Jaume Collet-Serra


In [23]:
x = df["Cast and crew"][0]
print(x)

Adam Robitel (director); Leigh Whannell (screenplay); Lin Shaye, Angus Sampson, Leigh Whannell, Spencer Locke, Caitlin Gerard, Kirk Acevedo, Bruce Davison


In [24]:
act = x.split(";")[-1].split(",")
print(f"{act[0]}  {act[1]}  {act[2]}")

 Lin Shaye   Angus Sampson   Leigh Whannell


In [25]:
def actors(x,num):
#     act = x.split(";")[-1].split(",")
#     act = (x.split("screenplay); ")[-1]).split(", ")
    casts = (x.split("screenplay); ")[-1]).split(", ")
    if casts == []:
        return np.NaN
    if num == 1:
        return (casts[0])
    elif num ==2:
        if len(casts)<=1:
            return np.NaN
        return (casts[1])
    elif num ==3:
        if  len(casts)<=2:
            return np.NaN
        else:
            return (casts[2])


In [27]:
df["actor_1_name"] = df["Cast and crew"].map(lambda x: actors(x,1))
df["actor_2_name"] = df["Cast and crew"].map(lambda x: actors(x,2))
df["actor_3_name"] = df["Cast and crew"].map(lambda x: actors(x,3))

In [28]:
df.drop("Cast and crew", axis=1, inplace =True)

In [29]:
df.tail(10)

,Title,genres,director_name,actor_1_name,actor_2_name,actor_3_name
501,Seberg,Thriller Drama,Benedict Andrews,Kristen Stewart,Jack O'Connell,Margaret Qualley
502,A Hidden Life,Drama War History,Terrence Malick,August Diehl,Valerie Pachner,Matthias Schoenaerts
503,Star Wars: The Rise of Skywalker,Action Adventure Science Fiction,J. J. Abrams,Carrie Fisher,Mark Hamill,Adam Driver
504,Cats,Fantasy Comedy Drama,Tom Hooper,James Corden,Judi Dench,Jason Derulo
505,Togo,Adventure Family,Ericson Core,Willem Dafoe,Julianne Nicholson,Christopher Heyerdahl
506,Spies in Disguise,Animation Action Adventure Comedy Family,"Nick Bruno, Troy Quane",Will Smith,Tom Holland,Rashida Jones
507,Little Women,Drama Romance,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh
508,1917,War Drama Action Thriller,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong
509,Just Mercy,Drama Crime History,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson
510,Clemency,Drama,Chinonye Chukwu,Alfre Woodard,Wendell Pierce,Aldis Hodge


In [30]:
reqAttr = pd.read_csv("Dataset/reqAttr.csv")
reqAttr.head()

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,avatar,Action Adventure Fantasy Sci-Fi,James Cameron,CCH Pounder,Joel David Moore,Wes Studi
1,pirates of the caribbean: at world's end,Action Adventure Fantasy,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport
2,spectre,Action Adventure Thriller,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman
3,the dark knight rises,Action Thriller,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt
4,star wars: episode vii - the force awakens ...,Documentary,Doug Walker,Doug Walker,Rob Walker,unknown


In [31]:
df = df.rename(columns= {"Title":"movie_title" })

In [32]:
df["movie_title"] = df["movie_title"].apply(lambda x: x.lower())

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Insidious: The Last Key,Horror Mystery Thriller,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell
1,The Strange Ones,Thriller Drama,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus
2,Stratton,Action Thriller,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan
3,Sweet Country,Drama History Western,Warwick Thornton,Bryan Brown,Sam Neill,NaN
4,The Commuter,Action Thriller,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson


In [36]:
df.head()

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,insidious: the last key,Horror Mystery Thriller,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell
1,the strange ones,Thriller Drama,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus
2,stratton,Action Thriller,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan
3,sweet country,Drama History Western,Warwick Thornton,Bryan Brown,Sam Neill,NaN
4,the commuter,Action Thriller,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson


#### Append the new dataframe with csv conatining movies from 1916 movies

In [37]:
reqAttr = reqAttr.append(df, ignore_index=True)

In [38]:
for i in reqAttr.columns:
    print(f"{i} =|=====|= {reqAttr[i].isnull().sum()}")

movie_title =|=====|= 0
genres =|=====|= 4
director_name =|=====|= 0
actor_1_name =|=====|= 0
actor_2_name =|=====|= 10
actor_3_name =|=====|= 41


In [39]:
reqAttr = reqAttr.dropna(how='any')

In [40]:
for i in reqAttr.columns:
    print(f"{i} =|=====|= {reqAttr[i].isnull().sum()}")

movie_title =|=====|= 0
genres =|=====|= 0
director_name =|=====|= 0
actor_1_name =|=====|= 0
actor_2_name =|=====|= 0
actor_3_name =|=====|= 0


In [41]:
reqAttr.to_csv("Dataset/reqAttr.csv", index=False)